In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/refined_model_dataset.csv')

In [3]:
df

,Date,Geography,Year,Quarter,Month,Season,Manufacturer Name,Product Name,Category,Flavor / Scent,Form,Package Size,Portion,Unit Sales,Volume Sales,Dollar Sales,Price per Unit
0,2020-01-01,California,2020,1,1,Winter,BEYOND MEAT INC,FROZEN FEISTY MEAT SUBSTITUTE CRUMBLE 10 OZ,FROZEN MEAT,SPICY,CRUMBLE,10.0,Small,2883.544343,1802.215214,14153.270715,4.908290
1,2020-01-01,Mid-South,2020,1,1,Winter,KELLANOVA,FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE BURGER...,FROZEN MEAT,VEGETABLE,BURGER,9.5,Small,9813.363484,5827.175237,36702.803983,3.740084
2,2020-01-01,Mid-South,2020,1,1,Winter,KELLANOVA,FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE BURGER...,FROZEN MEAT,VEGETABLE,BURGER,56.0,Large,223.000000,780.500000,3535.380000,15.853722
3,2020-01-01,Mid-South,2020,1,1,Winter,KELLANOVA,FROZEN FALAFEL FALAFEL BURGER 16 OZ,FROZEN FALAFEL,SOUR,BURGER,16.0,Medium,225.000000,225.000000,1642.230000,7.298800
4,2020-01-01,Mid-South,2020,1,1,Winter,KELLANOVA,FROZEN CHORIZO MEAT SUBSTITUTE CRUMBLE 10 OZ,FROZEN MEAT,PORK,CRUMBLE,10.0,Small,3199.886009,1999.928756,12043.475816,3.763720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383365,2023-12-01,South Central,2023,4,12,Winter,CONAGRA BRANDS,FROZEN REGULAR MEAT SUBSTITUTE BURGER 8 OZ,FROZEN MEAT,REGULAR,BURGER,8.0,Small,948.466720,474.233360,4631.711155,4.883367
383366,2023-12-01,South Central,2023,4,12,Winter,CONAGRA BRANDS,FROZEN REGULAR MEAT SUBSTITUTE LINK 14 OZ,FROZEN MEAT,REGULAR,LINK,14.0,Medium,698.368059,611.072052,5605.002443,8.025857
383367,2023-12-01,South Central,2023,4,12,Winter,CONAGRA BRANDS,FROZEN SPICY ITALIAN MEAT SUBSTITUTE PIECE 14 OZ,FROZEN MEAT,SPICY,PIECE,14.0,Medium,8.161198,7.141048,43.529406,5.333703
383368,2023-12-01,South Central,2023,4,12,Winter,CONAGRA BRANDS,FROZEN REGULAR MEAT SUBSTITUTE GROUND 13.7 OZ,FROZEN MEAT,REGULAR,GROUND,13.7,Medium,8879.623875,7603.621924,44725.400207,5.036858


## Lasso Regression Model

In [5]:
X = ['Geography', 'Year', 'Manufacturer Name', 'Category', 'Flavor / Scent', 'Form', 'Portion'] # add month or year, geography, ACV
#X = ['Geography', 'Year', 'Manufacturer Name', 'Category', 'Flavor / Scent', 'Form', 'Portion'] # add month or year, geography, ACV
#X = ['Geography', 'Portion', 'Flavor / Scent', 'Geography * Portion', 'Geography * Flavor / Scent', 'Year',' Month', 'Manufacturer Name', 'Category',' Form', 'ACV']
y = ['Dollar Sales']

categorical_features = X
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

lasso = Lasso(alpha=0.1)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', lasso)])

X_data = df[X]
y_data = df[y]

pipeline.fit(X_data, y_data)
y_pred = pipeline.predict(X_data)

print("Mean Squared Error:", mean_squared_error(y_data, y_pred))

# Coefficients of the model
feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)

# Coefficients of the model
coefficients = pipeline.named_steps['regressor'].coef_

# Combine feature names and coefficients into a DataFrame for better readability
feature_coefficients = pd.DataFrame(zip(feature_names, coefficients), columns=['Feature', 'Coefficient'])
feature_coefficients.head(50)

Mean Squared Error: 305848644.62518364


/Users/fuqiaozhi/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2128941354236.25, tolerance: 16261877052.9573
  model = cd_fast.sparse_enet_coordinate_descent(


,Feature,Coefficient
0,Geography_California,697.547097
1,Geography_Great Lakes,-74.761096
2,Geography_Mid-South,579.578189
3,Geography_Northeast,4599.351491
4,Geography_Plains,-6872.728748
5,Geography_South Central,-4536.935804
6,Geography_Southeast,1966.911908
7,Geography_West,-0.000000
8,Year_2020,3442.646006
9,Year_2021,1637.262478


In [6]:
feature_coefficients[feature_coefficients['Coefficient'] > 0]

,Feature,Coefficient
0,Geography_California,697.547097
2,Geography_Mid-South,579.578189
3,Geography_Northeast,4599.351491
6,Geography_Southeast,1966.911908
8,Year_2020,3442.646006
9,Year_2021,1637.262478
10,Year_2022,122.527352
12,Manufacturer Name_BEYOND MEAT INC,22407.939580
13,Manufacturer Name_CONAGRA BRANDS,7001.954268
14,Manufacturer Name_IMPOSSIBLE FOODS INC,25547.276070


In [7]:
feature_coefficients[feature_coefficients['Coefficient'] < 0]

,Feature,Coefficient
1,Geography_Great Lakes,-74.761096
4,Geography_Plains,-6872.728748
5,Geography_South Central,-4536.935804
11,Year_2023,-1604.180501
16,Manufacturer Name_LIGHTLIFE FOODS INC,-3760.539428
17,Manufacturer Name_PRIVATE LABEL,-4426.328945
18,Manufacturer Name_TURTLE ISLAND FOODS INC,-3912.172956
20,Category_FROZEN FALAFEL,-16288.953427
22,Category_FROZEN MEAT,-2554.546724
24,Category_FROZEN POULTRY,-2734.036655


In [8]:
r_squared_train = pipeline.score(X_data, y_data)
print("R-squared on Training Set:", r_squared_train)

R-squared on Training Set: 0.2789688760521253


## Dtree Model

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


In [10]:
# Define the feature columns and target column
features = ['Geography', 'Year', 'Manufacturer Name', 'Category', 'Flavor / Scent', 'Form', 'Portion']
target = 'Dollar Sales'

# Preprocessing for categorical data
categorical_features = features
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create preprocessing and training pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a Decision Tree regression model within a pipeline
decision_tree = DecisionTreeRegressor(random_state=0)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', decision_tree)])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.3, random_state=0)

# Fit the Decision Tree model
pipeline.fit(X_train, y_train)

# Evaluate the model on the testing set
y_pred_test = pipeline.predict(X_test)
print("Mean Squared Error on Test Set:", mean_squared_error(y_test, y_pred_test))
print("R-squared on Test Set:", pipeline.score(X_test, y_test))

# You can get the feature importances (which features are most important) like this:
feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_importances = pipeline.named_steps['regressor'].feature_importances_

# Combine feature names and importances into a DataFrame for better readability
feature_importances_df = pd.DataFrame(zip(feature_names, feature_importances), columns=['Feature', 'Importance'])

Mean Squared Error on Test Set: 77799212.71370786
R-squared on Test Set: 0.8160724354795967


In [11]:
new_importance_df = feature_importances_df.sort_values('Importance', ascending = False)
new_importance_df

,Feature,Importance
12,Manufacturer Name_BEYOND MEAT INC,8.232930e-02
90,Portion_Small,7.253699e-02
26,Category_REFRIGERATED MEAT,6.666986e-02
4,Geography_Plains,5.839927e-02
51,Form_BURGER PATTY,5.594356e-02
...,...,...
73,Form_RIBLET,1.430210e-08
72,Form_PULLED,2.300632e-10
77,Form_SAUSAGE PATTY,0.000000e+00
32,Flavor / Scent_FISH,0.000000e+00


## Pasty

In [12]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from patsy import dmatrices

formula = 'Q("Dollar Sales") ~ Q("Geography") + Q("Portion") + Q("Geography"):Q("Portion") + Q("Geography"):Q("Flavor / Scent") + Q("Flavor / Scent") + Q("Year") + Q("Month") + Q("Manufacturer Name") + Q("Category") + Q("Form")'

Y, X = dmatrices(formula, df, return_type='dataframe')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

lasso = LassoCV(cv=5, random_state=42)

lasso.fit(X_train, Y_train.values.ravel())

Y_pred = lasso.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean Squared Error: {mse}')



Mean Squared Error: 288595008.2891735


In [13]:
r_squared = lasso.score(X_test, Y_test)
print(f'R-squared: {r_squared}')

R-squared: 0.27945882111528764


In [14]:
feature_coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso.coef_
})
coef = feature_coef_df[feature_coef_df['Coefficient'] > 0]


In [15]:
coef = feature_coef_df[feature_coef_df['Coefficient'] < 0]
coef

,Feature,Coefficient
1,"Q(""Geography"")[T.Great Lakes]",-1361.713920
4,"Q(""Geography"")[T.Plains]",-4444.848072
7,"Q(""Geography"")[T.West]",-926.299182
10,"Q(""Flavor / Scent"")[T.BEEF]",-5403.467098
11,"Q(""Flavor / Scent"")[T.CHICKEN]",-2807.752473
...,...,...
157,"Q(""Geography"")[T.Southeast]:Q(""Flavor / Scent""...",-982.764576
158,"Q(""Geography"")[T.West]:Q(""Flavor / Scent"")[T.S...",-681.973814
165,"Q(""Geography"")[T.West]:Q(""Flavor / Scent"")[T.V...",-1547.793433
166,"Q(""Year"")",-1705.008207
